# Configure Notebook

In [ ]:
%matplotlib notebook
%pylab
# %matplotlib inline
%load_ext autoreload
%autoreload 2
%config InlineBackend.figure_format = 'retina'

**Cloning Xana from GitLab**

What you probably have just done.

In [ ]:
!git clone https://git.xfel.eu/gitlab/reiserm/Xana.git

**unpacking example data**

In [ ]:
!tar xzf ./xpcs_data.tgz

**Importing Analysis Software - Xana**

Working on _setup.py_. Later the Python-Path has not to be modified and Xana can just be imported.

In [ ]:
INSTALL_DIR = '../'

import os, sys
sys.path.append(os.path.expanduser(INSTALL_DIR + '/Xana'))

## Importing Xana

In [ ]:
from Xana import Xana

# Basic Example

The example data folder contains three time series of 100nm (diameter) SiO2 nano particles dispersed in a glycerol water mixture. One data set shows indications of beam damage--the correlation functions are shifted to faster relaxation times. 

The task is to:
* calculate correlation functions of all three data sets, 
* identify the _damaged_ data set
* determine the viscosity of the solution

### Creating A Setup File

For each type of analysis a _setup_-file is needed, that contains information about the geometry (sample-detector distance, wavelength, etc.) but also about the ROIs where the analysis should be performed.

In [ ]:
d = Xana(fmtstr='id10_eiger_single_edf', # format string that defines how the date are read
         sample='100nm SiO2 particles in glycerol', # optional: sample name used in data base
         maskfile='./masks/mask_structure_03.npy') # file that conatains the blemish / mask 
                                                   # (optional: but necessary for analysis)

Connecting Xana with a data folder allows for receiving meta data information on all data sets in that folder. 

In [ ]:
d.connect('./dukes100nm_glycerol/xpcs/exp320us_att0/')
d.meta

After the connection to a dataset&mdash;or a data folder with more than one dataset&mdash;has been established, the data can be accessed. In the most simple case, a 3D-array is returned (time x Pixel-X x Pixel-Y). For demonstration purposes, we read only the first 100 images by setting the last kwarg to (100,).

In [ ]:
data = d.get_series(0, method='full', verbose=False, last=(100,))
print('\nThe returned array has the dimensions: ', data.shape)

For setting the ROIs properly, it would be convenient to have a time averaged image. The command is the same except the method kwarg has been changed to _average_. Now the command will return a list containing the average intensities, I , and the variance over time, V.

In [ ]:
I, V = d.get_series(0, method='average', verbose=False, last=(100,))

Now the setup can be defined. If no dict is passed to _make_setup_, the user is asked for the specific arguments. The vector qv_init contains a list of tuples. Each tuple should have the q position (or an array of q positions) as first element and the q width as second argument.

**defineqrois** needs an image, I, and the qv_init list as input parameters. The plot argument can [0,1,2], d can be between 10 and 100000. Try what happens when you play with those parameters. Try also to add the phiv_init kwarg (at the moment commented).

In [ ]:
d.make_setup(**{'ctr': [516, 257], 'lambda': 1.5,
                'pix_size': (75, 75), 'distance': 5})

qv_init = [(np.arange(0.025, 0.08, .01), .008),
           (.125, .02)]
phiv_init = [([10, 150], 30)]

d.defineqrois(I, qv_init=qv_init, plot=2, d=300,) # phiv_init=phiv_init,)

If you are content with the setup&mdash;try to stick to the default values&mdash;you can save it. The maksavdir command creates a new subfolder _setupfiles_ in the folder specified by the second argument, here in the present folder ./ . Prefix and suffix are added automatically.

In [ ]:
d.mksavdir('setupfiles', './')
d.save_setup('100nm')

# Calculate Correlation Functions

Now that we have a setup file, we can proceed with the actual analysis. The setup you created is already part of Xana now. If you start from this point, you can recreate the Xana object with the setupfile as a kwarg:

In [ ]:
d = Xana(fmtstr='id10_eiger_single_edf', # format string that defines how the date are read
         sample='100nm SiO2 particles in glycerol', # optional: sample name used in data base
         setupfile='./setupfiles/setup_100nm.pkl', # optional: 
         maskfile='./masks/mask_structure_03.npy') # file that conatains the blemish / mask 
                                                   # (optional: but necessary for analysis)

... or load the setupfile:

In [ ]:
d.load_setup('/Users/reiserm/data/setupfiles/setup_100nm.pkl')

In [ ]:
d.connect('./dukes100nm_glycerol/xpcs/exp320us_att0/')
d.meta

we need a new folder for the analysis. Again we can use the mksavdir command

In [ ]:
d.mksavdir('100nm_glycerol', './')

The results of the analysis are stored (pickled) as dictionaries containing all necessary information on the analysis. Additionally, a database entry is added in the analysis database. The default file is Analysis_db.pkl. If that file does not exist, the database has to be initialized **after** connecting with a dataset. That is because the columns of the database are taken from the meta data table.

In [ ]:
d.init_db()

Different types of analysis can be performed as shown in the next cell. The command is allways analyze().
* The first argument is the index of the meta data table and defines the series you want to analyze. -1 means that every series is analyzed (here index [0,1,2])
* the second argument defines the type of analysis; **xpcs** uses pyxpcs, **xpcs_evt** uses the eventcorrelator
* other kwargs are specific for that type of analysis

In [ ]:
d.analyze(-1, 'saxs', verbose=True,)
d.analyze(-1, 'xpcs', verbose=True, twotime_par=1, nprocs=4, nread_procs=1)
d.analyze(-1, 'xpcs_evt', verbose=True, nprocs=4, nread_procs=1)
d.analyze(-1, 'xsvs', verbose=True, nbins=64, nprocs=4, nread_procs=1)

# Data Interpretation

For plotting the results we need an Xana object and to load the analysis database. The Xana instance is then passed to Soq, CorrFunc or VisbFunc for data visualization and fitting. But that is a story for another day...

## Loading Data Base

In [ ]:
d = Xana(setupfile='./setupfiles/setup_100nm.pkl',
         maskfile='./masks/mask_structure_03.npy')
d.load_db('./100nm_glycerol/Analysis_db.pkl')

## SAXS

In [ ]:
from SaxsAna.Soq import Soq

In [ ]:
s = Soq(d)
s.plot_soq([0,1,3], Ae=0,)

## XPCS

In [ ]:
from XpcsAna.CorrFunc import CorrFunc

In [ ]:
# initializing CorrFunc instance
g = CorrFunc(d)

# searching data base for xpcs measurements
ind = d.db[(d.db['analysis']=='xpcs')
          &(d.db['use'])].index.values

#loading correlation functions
g.get_g2(ind,)

# plotting
g.nq = np.arange(3)
g.plot_g2(doplot='data')

In [ ]:
d.discard_entry(3)

In [ ]:
g = CorrFunc(d)
ind = d.db[(d.db['analysis']=='xpcs')
          &(d.db['use'])].index.values
g.get_g2(ind, merge='merge')
g.nq = np.arange(3)
g.plot_g2(doplot='data_fit_legd', 
          change_marker=1, markersize=5, alpha=.6, # style options
          fix={'g0':1,}, dofit=True,)

In [ ]:
g.rescale(normby='fit', norm_contrast=True, contrast=1, baseline=0)

In [ ]:
g.plot_g2(doplot='data_fit_legd', change_marker=1, markersize=5, alpha=.6, dofit=True, 
          fix={'g0':1, },)# init={'beta':(1,0,1), 't0':(.01,None,None), 'g0':(1,None,None)})

In [ ]:
ge = CorrFunc(d)
ind = d.db[(d.db['analysis']=='xpcs_evt')
          &(d.db['use'])].index.values
ge.get_g2(ind, merge='merge')
ge.nq = np.arange(1,2)
ge.plot_g2(doplot='data_fit_legd', change_marker=1, markersize=5, alpha=.6, dofit=True, 
          fix={'g0':1,'beta':.21}, mode='semilogx', color_mode=0)

In [ ]:
ge = CorrFunc(d)
ind = d.db[(d.db['analysis']=='xpcs_evt')
          &(d.db['use'])].index.values
ge.get_g2(ind, merge='merge')
# ge.nq = np.arange(3)
ge.merge_g2list(resample=120)
ge.rescale(normby='average', interval=(0,-20), weighted=False)
ge.plot_g2(nq=np.arange(3), doplot='data_fit_legd', change_marker=1, markersize=5, alpha=.6, dofit=True, 
          fix={'g0':1, 'a':1}, mode='semilogx', color_mode=0, init={'t0':(0.02, None, None)})

In [ ]:
gs = g + ge

In [ ]:
gs.plot_parameters(['G', 'kww'], 
                  modes=([1],[1]),
                   alpha=(2,1),
                   fit=('lin',''),
                   fix=({},{}),
#                    log=('',''),
        
                   cmap='tab10', R=50e-9, viscosity=1, 
                   fit_report=False,);


In [ ]:
g.plot_trace(4)

In [ ]:
g.plot_twotime(4, interpolation='nearest',)

## XSVS

In [ ]:
from XsvsAna.VisbFunc import VisbFunc

In [ ]:
# initializing Visibility Func instance
v = VisbFunc(d)

# searching data base for xsvs results
ind = d.db[(d.db['analysis']=='xsvs')
          &(d.db['use'])].index.values

# loading probablities
v.get_prob(ind, merge=True)

# calculating contrast
v.calculate_contrast()

# plot histogram
v.quicklook(plot=['pbb'], nq=np.arange(5), color_mode=0, ratio=0, cmap='tab10', alpha=0.2)  

In [ ]:
v.quicklook(plot=['pbb','bvkb', 'kbvi', 'bvi', 'pkvkb', 'pbk'],
                      idx=None, nq=np.arange(5), change_marker=0,
                       color_mode=1, maxlen=np.inf, ratio=0, probk=2,
                       cmap='tab10', lfs=8, total_counts=False, alpha=0.2)